In [9]:
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

Fill mising value

In [10]:
train_data = pd.read_csv('Train_samsung.csv')


def imputing_data(data):
    numeric_columns = data.select_dtypes(include='number').columns
    categorical_columns = data.select_dtypes(exclude='number').columns

    imputer_numeric = SimpleImputer(strategy='mean')
    imputer_categorical = SimpleImputer(strategy='most_frequent')

    data[numeric_columns] = imputer_numeric.fit_transform(data[numeric_columns])
    data[categorical_columns] = imputer_categorical.fit_transform(data[categorical_columns])

    label_encoder = LabelEncoder()
    for column in categorical_columns:
        data[column] = label_encoder.fit_transform(data[column])

    data[numeric_columns] = data[numeric_columns].round(1)

    return data


imputing_data(train_data)
display(train_data)

# header = ','.join(train_data.columns)
# train_data.to_csv('Filled_train.csv', index=False, header=header)

raw_test_data = pd.read_csv('Test_samsung_noclass.csv')
imputing_data(raw_test_data)
display(raw_test_data)
#
# header2 = ','.join(raw_test_data.columns)
# raw_test_data.to_csv('Filled_test.csv', index=False, header=header2)


,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,Class
0,1,1,0,0,0,6000.0,2250.0,265.0,360.0,0.8,1,0
1,1,1,0,0,0,2958.0,2900.0,131.0,360.0,1.0,1,1
2,1,1,2,0,0,6250.0,1695.0,210.0,360.0,1.0,1,1
3,1,1,0,0,0,2083.0,3150.0,128.0,360.0,1.0,1,1
4,1,0,0,0,0,4166.0,0.0,98.0,360.0,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
486,1,1,2,1,1,1875.0,1875.0,97.0,360.0,1.0,1,1
487,1,1,2,0,0,11417.0,1126.0,225.0,360.0,1.0,2,1
488,0,0,0,0,0,3237.0,0.0,30.0,360.0,1.0,2,1
489,0,1,0,0,0,10047.0,0.0,148.7,240.0,1.0,1,1


,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11
0,1,1,0,0,0,9083.0,0.0,228.0,360.0,1.0,1
1,1,1,0,0,0,4310.0,0.0,130.0,360.0,0.8,1
2,1,1,2,0,0,4167.0,1447.0,158.0,360.0,1.0,0
3,0,0,0,0,0,2900.0,0.0,71.0,360.0,1.0,0
4,1,0,0,0,0,2718.0,0.0,70.0,360.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...
118,1,1,0,0,0,3716.0,0.0,42.0,180.0,1.0,0
119,0,0,0,0,0,2507.0,0.0,56.0,360.0,1.0,0
120,1,1,1,0,0,2882.0,1843.0,123.0,480.0,1.0,1
121,1,1,2,0,0,2500.0,1840.0,109.0,360.0,1.0,2


Separating features and labels

In [11]:
# processed_data = pd.read_csv('Filled_train.csv')

X_train = train_data.drop('Class', axis=1)
y_train = train_data['Class']

X_train, y_train

(     X1  X2  X3  X4  X5       X6      X7     X8     X9  X10  X11
 0     1   1   0   0   0   6000.0  2250.0  265.0  360.0  0.8    1
 1     1   1   0   0   0   2958.0  2900.0  131.0  360.0  1.0    1
 2     1   1   2   0   0   6250.0  1695.0  210.0  360.0  1.0    1
 3     1   1   0   0   0   2083.0  3150.0  128.0  360.0  1.0    1
 4     1   0   0   0   0   4166.0     0.0   98.0  360.0  0.0    1
 ..   ..  ..  ..  ..  ..      ...     ...    ...    ...  ...  ...
 486   1   1   2   1   1   1875.0  1875.0   97.0  360.0  1.0    1
 487   1   1   2   0   0  11417.0  1126.0  225.0  360.0  1.0    2
 488   0   0   0   0   0   3237.0     0.0   30.0  360.0  1.0    2
 489   0   1   0   0   0  10047.0     0.0  148.7  240.0  1.0    1
 490   1   1   0   0   0  13650.0     0.0  148.7  360.0  1.0    2
 
 [491 rows x 11 columns],
 0      0
 1      1
 2      1
 3      1
 4      0
       ..
 486    1
 487    1
 488    1
 489    1
 490    1
 Name: Class, Length: 491, dtype: int32)

Build discriminative model, Predict_test.csv is tested file with Class label

In [12]:
# X_test = pd.read_csv('Filled_test.csv')
X_test = raw_test_data.copy()

model = RandomForestClassifier()
model.fit(X_train, y_train)

y_predict = model.predict(X_test)
label_mapping = {0: 'N', 1: 'Y'}


X_test["Class"] = y_predict
X_test["Class"] = X_test["Class"].replace(label_mapping)

X1_mapping ={0:'Female', 1:'Male'}
X_test["X1"] = X_test["X1"].replace(X1_mapping)

X2_mapping ={0:'No', 1:'Yes'}
X_test["X2"] = X_test["X2"].replace(X2_mapping)

X3_mapping ={0:'0', 1:'1', 2:'2', 3:'3+'}
X_test["X3"] = X_test["X3"].replace(X3_mapping)

X4_mapping ={0:'Graduate', 1:'Not Graduate'}
X_test["X4"] = X_test["X4"].replace(X4_mapping)

X5_mapping ={0:'No', 1:'Yes'}
X_test["X5"] = X_test["X5"].replace(X5_mapping)



X_test.to_csv("Predict_test.csv",)
X_test


,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,Class
0,Male,Yes,0,Graduate,No,9083.0,0.0,228.0,360.0,1.0,1,Y
1,Male,Yes,0,Graduate,No,4310.0,0.0,130.0,360.0,0.8,1,Y
2,Male,Yes,2,Graduate,No,4167.0,1447.0,158.0,360.0,1.0,0,Y
3,Female,No,0,Graduate,No,2900.0,0.0,71.0,360.0,1.0,0,Y
4,Male,No,0,Graduate,No,2718.0,0.0,70.0,360.0,1.0,1,Y
...,...,...,...,...,...,...,...,...,...,...,...,...
118,Male,Yes,0,Graduate,No,3716.0,0.0,42.0,180.0,1.0,0,Y
119,Female,No,0,Graduate,No,2507.0,0.0,56.0,360.0,1.0,0,N
120,Male,Yes,1,Graduate,No,2882.0,1843.0,123.0,480.0,1.0,1,Y
121,Male,Yes,2,Graduate,No,2500.0,1840.0,109.0,360.0,1.0,2,Y
